In [ ]:
# Normal one, with FPS

import cv2
import mediapipe as mp
import imutils
import time

# Initialize MediaPipe objects for hand tracking and drawing
mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

# Function to process an input image and return the results of hand tracking
def process_image(img):
    # Convert the input image to RGB
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Run hand tracking on the image
    results = hands.process(gray_image)
    return results

# Function to draw connections between hand landmarks
def draw_hand_connections(img, results):
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            for id, lm in enumerate(handLms.landmark):
                # Get the height and width of the image
                h, w, c = img.shape
                # Compute the (x, y) coordinates of the hand landmark
                cx, cy = int(lm.x * w), int(lm.y * h)
                # Draw a circle at the hand landmark
                cv2.circle(img, (cx, cy), 10, (0, 255, 0), cv2.FILLED)
            # Draw connections between hand landmarks
            mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)
        return img

def main():
    # Open a VideoCapture object to access the webcam
    cap = cv2.VideoCapture(0)
    
    # Initialize variables to track FPS
    start_time = time.time()
    fps = 0
    frame_count = 0
    
    while True:
        # Capture a frame from the webcam
        success, image = cap.read()
        # Resize the image to 1024x1024
        image = imutils.resize(image, width=1024, height=1024)
        # Run hand tracking on the image
        results = process_image(image)
        # Draw connections between hand landmarks
        draw_hand_connections(image, results)

        # Update frame count for FPS calculation
        frame_count += 1
        # Flip the image horizontally
        image = cv2.flip(image, 1)  
    
        # Calculate FPS once per second
        if time.time() - start_time > 1:
            fps = frame_count / (time.time() - start_time)
            frame_count = 0
            start_time = time.time()

        # Display the number of hands detected
        if results.multi_hand_landmarks:
            num_hands = len(results.multi_hand_landmarks)
            cv2.putText(image, f'Number of Hands: {num_hands}', (10, 20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        # Display FPS on the Image
        cv2.putText(image, f'FPS: {fps:.0f}', (10, 40),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        cv2.imshow("Hand tracker", image)

        # Wait for 1 millisecond and check if the 'q' key is pressed
        # If 'q' is pressed, release the video capture object and close all windows
        if cv2.waitKey(1) == ord('q'):
            cap.release()
            cv2.destroyAllWindows()

# Run function
if __name__ == "__main__":
    main()

In [ ]:
# Hand Tracking [Colored]
import cv2
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

# For webcam input:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
  while cap.isOpened():
    success, image = cap.read() 
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)

    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('Hand Tracking [Colored]', cv2.flip(image, 1))
    if cv2.waitKey(1) & 0xFF == ord('q'):
      break
cap.release()
cv2.destroyAllWindows()


In [ ]:
# Whole Body Capture
import cv2
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic

# For webcam input:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as holistic:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = holistic.process(image)

    # Draw landmark annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.face_landmarks,
        mp_holistic.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp_drawing_styles
        .get_default_face_mesh_contours_style())
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles
        .get_default_pose_landmarks_style())
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Holistic', cv2.flip(image, 1))
    if cv2.waitKey(1) & 0xFF == ord('q'):
      break
cap.release()
cv2.DestroyAllWindows()
